In [4]:
import os
import json
import pandas as pd
import traceback

In [5]:
from langchain_openai import ChatOpenAI

In [6]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [7]:
KEY=os.getenv("OPENAI_API_KEY")

In [9]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [10]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001F1E11DCBC0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001F1E11DE3C0>, temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='')

In [11]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [12]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [13]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [14]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [15]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [16]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [17]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [18]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [19]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [23]:
file_path=r"C:\Users\sahil pandey\genq.txt"

In [24]:
file_path

'C:\\Users\\sahil pandey\\genq.txt'

In [25]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [26]:
print(TEXT)

India is a federal republic comprising 28 states and 8 union territories. The states and union territories are further subdivided into districts and smaller administrative divisions.

The states are self-governing administrative divisions, each having a state government. The governing powers of the states are shared between the state government and the union government. The union territories are directly governed by the union government though some of the union territories have their own territorial government with certain autonomy.

The framework for the states and union territories were established by the Constitution of India, which came into effect on 26 January 1950. As per the Indian Constitution, the power to demarcate states and union territories in India is solely reserved with the Parliament of India, which can form new states/union territories, separate/merge territories from/to an existing state or convert states to union territories or vice versa.

History
British Raj: bef

In [27]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [28]:
NUMBER=5 
SUBJECT="Indian states"
TONE="simple"

In [29]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

c:\Users\sahil pandey\mcqgenr\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:India is a federal republic comprising 28 states and 8 union territories. The states and union territories are further subdivided into districts and smaller administrative divisions.

The states are self-governing administrative divisions, each having a state government. The governing powers of the states are shared between the state government and the union government. The union territories are directly governed by the union government though some of the union territories have their own territorial government with certain autonomy.

The framework for the states and union territories were established by the Constitution of India, which came into effect on 26 January 1950. As per the Indian Constitution, the power to demarcate states and union territories in India is solely reserved with the Parliament of India, which can form new states/union territories, separate/merge territori

In [30]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1979
Prompt Tokens:1284
Completion Tokens:695
Total Cost:0.0033160000000000004


In [31]:
response

{'text': 'India is a federal republic comprising 28 states and 8 union territories. The states and union territories are further subdivided into districts and smaller administrative divisions.\n\nThe states are self-governing administrative divisions, each having a state government. The governing powers of the states are shared between the state government and the union government. The union territories are directly governed by the union government though some of the union territories have their own territorial government with certain autonomy.\n\nThe framework for the states and union territories were established by the Constitution of India, which came into effect on 26 January 1950. As per the Indian Constitution, the power to demarcate states and union territories in India is solely reserved with the Parliament of India, which can form new states/union territories, separate/merge territories from/to an existing state or convert states to union territories or vice versa.\n\nHistory\

In [32]:
quiz=response.get("quiz")

In [33]:
quiz=json.loads(quiz)

In [34]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [35]:
quiz_table_data

[{'MCQ': 'How many states and union territories does India comprise?',
  'Choices': 'a: 20 states and 10 union territories | b: 28 states and 8 union territories | c: 30 states and 6 union territories | d: 25 states and 5 union territories',
  'Correct': 'b'},
 {'MCQ': 'Who has the power to demarcate states and union territories in India?',
  'Choices': 'a: President of India | b: Prime Minister of India | c: Parliament of India | d: Supreme Court of India',
  'Correct': 'c'},
 {'MCQ': 'Which act led to the British Crown taking over the administration of India from the British East India Company?',
  'Choices': 'a: Government of India Act 1858 | b: Indian Rebellion Act 1857 | c: British Raj Act 1860 | d: Indian Administration Act 1862',
  'Correct': 'a'},
 {'MCQ': 'How many major provinces were there in India in 1907 during the British Raj?',
  'Choices': 'a: 5 | b: 7 | c: 10 | d: 12',
  'Correct': 'b'},
 {'MCQ': 'Who exercised suzerainty over the princely states in British India?',
  

In [36]:
quiz=pd.DataFrame(quiz_table_data)

In [37]:
quiz.to_csv("machinelearning.csv",index=False)

In [38]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'04_27_2024_01_55_02'